In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

data = pd.read_csv("SolarPrediction.csv")

# Drop unnecessary columns
data.drop(['Date', 'Time'], axis=1, inplace=True)

# Separate features and target variable
X = data.drop('Radiation', axis=1)
y = data['Radiation']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [2]:
from sklearn.neural_network import MLPRegressor

def create_ann():
    # Define the neural network structure
    ann = MLPRegressor(hidden_layer_sizes=(10, 10), max_iter=200, random_state=42)
    return ann


In [3]:
import numpy as np
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import warnings

# Suppress convergence warnings
warnings.filterwarnings("ignore")


# Define EO parameters
num_particles = 30
max_iterations = 100
alpha = 0.5
beta = 0.5
CP_1 = 1.0
LB = -1
UB = 1

# Initialize particles and concentrations
particles = np.random.uniform(LB, UB, (num_particles, X_train.shape[1]))
concentrations = np.array([LB + np.random.rand() * (UB - LB) for _ in range(num_particles)])

# Define the fitness function
def fitness_function(particle):
    ann = create_ann()
    ann.fit(X_train, y_train)
    predictions = ann.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    return mse

# Main EO loop with tqdm for progress tracking
for iteration in tqdm(range(max_iterations), desc='Iterations', unit='iteration'):
    fitness_values = []
    for i in range(num_particles):
        fitness = fitness_function(particles[i])
        fitness_values.append(fitness)
        
    # Sort particles by fitness and select the top ones
    sorted_indices = np.argsort(fitness_values)
    top_particles = particles[sorted_indices[:4]]
    eq_candidate = np.mean(top_particles, axis=0)
    
    # Update particles towards equilibrium state
    for i in range(num_particles):
        r1, r2, r3, r4 = np.random.rand(4)
        F = CP_1 * np.sign(r2 - 0.5) * (np.exp(-beta * iteration / max_iterations) - 1)
        G = r3 * np.sign(r4 - 0.5) * (eq_candidate - particles[i])
        particles[i] = eq_candidate + F * (alpha * G + beta * (eq_candidate - particles[i]))

# Train the final ANN with the best particle
best_particle_idx = np.argmin(fitness_values)
best_particle = particles[best_particle_idx]

final_ann = create_ann()
final_ann.fit(X_train, y_train)

# Evaluate the final model
final_predictions = final_ann.predict(X_test)
final_mse = mean_squared_error(y_test, final_predictions)
print("Final MSE:", final_mse)


Iterations:   0%|          | 0/100 [00:00<?, ?iteration/s]

/home/samarth5854/anaconda3/envs/PsycheSail/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/samarth5854/anaconda3/envs/PsycheSail/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/samarth5854/anaconda3/envs/PsycheSail/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/samarth5854/anaconda3/envs/PsycheSail/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the opti